# **GenoVarDis@IberLEF2024: Automatic Genomic Variants and Related Diseases using Named Entity Recognition with Large Language Models**

## Autor: Víctor Manuel Oliveros Villena


In [ ]:
# Esta es la ruta a la raíz de nuestro Drive.
# Si se prefiere otra ruta, añadir los directorios pertinentes a continuación de esta.
path_drive = '/content/drive/MyDrive'

### **Configuración del entorno**

In [ ]:
!pip install gliner transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Usin

In [ ]:
import numpy as np
import torch
import os
from google.colab import drive
import pandas as pd
from gliner import GLiNER
from transformers import BasicTokenizer
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## **Preprocesado**

### **Lectura de archivos**

In [ ]:
# Ruta al archivo TSV en Google Drive
path_train_text = path_drive + '/GenoVarDis/Data/train_text.tsv'
path_train_annotation = path_drive + '/GenoVarDis/Data/train_annotation.tsv'
path_dev_text = path_drive + '/GenoVarDis/Data/dev_text.tsv'

# Leer el archivo TSV utilizando pandas
train_text = pd.read_csv(path_train_text, sep='\t')
train_annotation = pd.read_csv(path_train_annotation, sep='\t')
dev_text = pd.read_csv(path_dev_text, sep='\t')

# **GenoVarDis@IberLEF2024: Automatic Genomic Variants and Related Diseases using Named Entity Recognition with Large Language Models**

## Autor: Víctor Manuel Oliveros Villena


### **Preprocesado para adaptar el formato de nuestros ejemplos al empleado por modelos GLiNER**

In [ ]:
basic_tokenizer = BasicTokenizer()
data = np.array([], dtype=dict)

def tokenize_and_set_ids(example):
  """
  Tokeniza el texto y asigna IDs y etiquetas NER basadas en anotaciones para un ejemplo dado.

  Args:
      example (dict): Un diccionario que contiene un caso clinico.
      type_set (str): Indica si el ejemplo es parte del conjunto de entrenamiento ('train') o validacion ('dev').

  Returns:
      pd.DataFrame: DataFrame actualizado con los tokens y etiquetas NER para el ejemplo.
  """
  global data
  indices = set()
  result = dict()

  pmid = example['pmid']    # Pmid del caso clinico
  text = example['text']    # Texto del caso clinico
  tokens = np.array(basic_tokenizer.tokenize(text)) # Tokeniza el texto
  tags = []

  # Toma de datos del archivo correspondiente segun la opcion escogida
  train_sort_df = train_annotation[train_annotation['pmid'] == pmid].sort_values(by='offset1')
  labels = train_sort_df['label'].values.astype(str)  # Tipos de entidades
  spans = train_sort_df['span'].values.astype(str)    # Entidades

  index = 0
  # Por cada entidad...
  for l, d in zip(labels, spans):
    span_split = np.array(basic_tokenizer.tokenize(d)) # Tokeniza la entidad
    # Busca todas las apariciones de la entidad en el caso clinico
    index = np.where(tokens == span_split[0])[0]
    found = False
    # Si la entidad solo es una palabra...
    if len(span_split) == 1:
      i = 0
      # Mientras que no encontremos el indice de aparicion correcto...
      while not found and i < len(index):
        # Si el indice es valido (no fue escogido previamente)...
        if index[i] not in indices and (len(indices) == 0 or index[i] > max(indices)):
          # Lo guardamos en la lista de indices asignados
          indices.add(index[i])
          # Asignamos el tipo de entidad segun el modelo GLiNER
          tags.append([index[i], index[i], l])
          # Marcamos que hemos encontrado un indice
          found = True
        i += 1
    # Si la entidad contiene varias palabras...
    else:
      k = 0
      # Mientras que no encontramos el indice de aparicion correcto...
      while not found and k < len(index):
        i = index[k]
        # Si el indice es valido (no fue escogido previamente)...
        if i not in indices and (len(indices) == 0 or i > max(indices)):
          # Si la totalidad de la entidad coincide...
          if np.array_equal(tokens[i:i+len(span_split)], span_split):
            # Marcamos los indices correspondientes como asignados
            indices.update(np.arange(i, i+len(span_split)))
            # Asignamos el tipo de entidad segun el modelo GLiNER
            tags.append([i, i+len(span_split)-1, l])
            found = True
        k += 1
  # Guardamos el resultado final en un formato compatible con los modelos GLiNER
  result['tokenized_text'] = tokens.tolist()
  result['ner'] = tags

  # Añadimos el resultado a nuestro dataframe
  data = np.append(data, result)

  return data.tolist()


In [ ]:
# Aplicamos la funcion anterior a la totalidad del conjunto de entrenamiento
train_text.apply(tokenize_and_set_ids, axis=1)

0      [{'tokenized_text': ['12672033', '|', 't', '|'...
1      [{'tokenized_text': ['12672033', '|', 't', '|'...
2      [{'tokenized_text': ['12672033', '|', 't', '|'...
3      [{'tokenized_text': ['12672033', '|', 't', '|'...
4      [{'tokenized_text': ['12672033', '|', 't', '|'...
                             ...                        
422    [{'tokenized_text': ['12672033', '|', 't', '|'...
423    [{'tokenized_text': ['12672033', '|', 't', '|'...
424    [{'tokenized_text': ['12672033', '|', 't', '|'...
425    [{'tokenized_text': ['12672033', '|', 't', '|'...
426    [{'tokenized_text': ['12672033', '|', 't', '|'...
Length: 427, dtype: object

In [ ]:
# Total de ejemplos: Ejemplos de entrenamiento
len(data)

427

## **Entrenamiento del fine-tuning de GLiNER Small**

In [ ]:
# Cargamos el modelo GLiNER concreto que vamos a emplear, el Small
model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

In [ ]:
from types import SimpleNamespace

# Hiperparametros del modelo
config = SimpleNamespace(
    num_steps=20,
    train_batch_size=2,
    eval_every=1,
    save_directory= path_drive + "/GenoVarDis/Logs",
    warmup_ratio=0.1,
    device='cpu',
    lr_encoder=1e-5,
    lr_others=5e-5,
    freeze_token_rep=False,

    max_types=25,
    shuffle_types=True,
    random_drop=True,
    max_neg_type_ratio=1,
    max_len=700
)

In [ ]:
def train(model, config, train_data, eval_data=None):
    """
    Entrena el modelo con los datos de entrenamiento y evalúa periódicamente con los datos de evaluación.

    Args:
        model: El modelo a entrenar.
        config: Configuración con los hiperparámetros de entrenamiento.
        train_data: Datos de entrenamiento.
        eval_data: Datos de evaluación.

    Returns:
        None
    """
    model = model.to(config.device)

    # Establece los hiperparametros del modelo
    model.set_sampling_params(
        max_types=config.max_types,
        shuffle_types=config.shuffle_types,
        random_drop=config.random_drop,
        max_neg_type_ratio=config.max_neg_type_ratio,
        max_len=config.max_len
    )

    # Modo entrenamiento
    model.train()

    # Inicializa los cargadores de datos
    train_loader = model.create_dataloader(train_data, batch_size=config.train_batch_size, shuffle=True)

    # Inicializa el optimizador
    optimizer = model.get_optimizer(config.lr_encoder, config.lr_others, config.freeze_token_rep)

    pbar = tqdm(range(config.num_steps)) # Barra de progreso para el entrenamiento

    # Calcula el numero de pasos de calentamiento
    if config.warmup_ratio < 1:
        num_warmup_steps = int(config.num_steps * config.warmup_ratio)
    else:
        num_warmup_steps = int(config.warmup_ratio)

    # Inicializa el scheduler con calentamiento y decaimiento cosenoidal
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=config.num_steps
    )

    iter_train_loader = iter(train_loader) # Iterador del cargador de datos

    for step in pbar:
        try:
            x = next(iter_train_loader) # Obtiene el siguiente batch
        except StopIteration:
            iter_train_loader = iter(train_loader) # Reinicia el iterador si llega el final
            x = next(iter_train_loader)

        # Mueve los datos al dispositivo configurado
        for k, v in x.items():
            if isinstance(v, torch.Tensor):
                x[k] = v.to(config.device)

        loss = model(x)  # Forward pass

        # Verifica si la perdida es NaN
        if torch.isnan(loss):
            continue

        loss.backward()  # Calculo los gradientes
        optimizer.step()  # Actualiza los parametros
        scheduler.step()  # Actualiza la programacion del learning rate
        optimizer.zero_grad()  # Resetea los gradientes

        # Actualiza la barra de progreso
        description = f"step: {step} | epoch: {step // len(train_loader)} | loss: {loss.item():.2f}"
        pbar.set_description(description)

        # Evalua el modelo periodicamente (conjunto de validacion)
        if (step + 1) % config.eval_every == 0:

            model.eval()

            if eval_data is not None:
                results, f1 = model.evaluate(eval_data["samples"], flat_ner=True, threshold=0.5, batch_size=12,
                                     entity_types=eval_data["entity_types"])

                print(f"Step={step}\n{results}")

            if not os.path.exists(config.save_directory):
                os.makedirs(config.save_directory)

            # Guarda el modelo
            if step == config.num_steps - 1:
              model.save_pretrained(f"{config.save_directory}/GLiNERSmall_DevTest")

            model.train()

In [ ]:
# Diccionario con informacion sobre el conjunto de validacion y tipos de entidades
eval_data = {
    "entity_types": ["Gene", "Disease", "DNAMutation", "SNP", "DNAAllele", "NucleotideChange-BaseChange", "OtherMutation", "Transcript"],
    "samples": data[:int(len(data)*0.1)]
}

# Entrenamiento del modelo
train(model, config, data, eval_data)

step: 0 | epoch: 0 | loss: 241.65:   0%|          | 0/20 [00:19<?, ?it/s]

Step=0
P: 47.16%	R: 32.25%	F1: 38.31%



step: 1 | epoch: 0 | loss: 210.94:   5%|▌         | 1/20 [03:46<1:04:52, 204.88s/it]

Step=1
P: 46.72%	R: 36.01%	F1: 40.67%



step: 2 | epoch: 0 | loss: 284.08:  10%|█         | 2/20 [07:03<59:59, 199.96s/it]

...

step: 18 | epoch: 0 | loss: 110.99:  90%|█████████ | 18/20 [1:00:27<06:45, 202.52s/it]

Step=18
P: 58.50%	R: 38.34%	F1: 46.32%



step: 19 | epoch: 0 | loss: 34.88:  95%|█████████▌| 19/20 [1:03:47<03:20, 200.58s/it] 

Step=19
P: 58.50%	R: 38.34%	F1: 46.32%



step: 19 | epoch: 0 | loss: 34.88: 100%|██████████| 20/20 [1:06:57<00:00, 200.89s/it]
